In [3]:
import os
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [49]:
i = 0
hub = 'LGA'
granularity = 5
for file in os.listdir('raw2018/'+hub):
    i += 1
    if '.csv' in file:
#         print(file)
        df = pd.read_csv('raw2018/'+hub+'/'+file,skiprows=6)
#         break
        df = df.dropna()
        df['Date'] = pd.to_datetime(df['Date (MM/DD/YYYY)'])
        df['Hour'] = df['Actual Arrival Time'].apply(lambda x:str(x).split(':')[0])
        df['Hour'] = df['Hour'].apply(lambda x: 0 if str(x)=='24' else x)
        df['Min'] = df['Actual Arrival Time'].apply(lambda x:int(str(x).split(':')[1]))
        df['Min'] = (df['Min'] / granularity).astype(int)*granularity
        df = df[['Date','Hour','Min']]
        df.dropna(inplace=True)
#         break
#         print(df.head(2))
        if i == 1:
            df.to_csv(str(granularity)+'min/'+hub+'.csv',index=False)
        elif i> 1:
            df.to_csv(str(granularity)+'min/'+hub+'.csv',mode='a+',header=None,index=False)

In [50]:
df = pd.read_csv(str(granularity)+'min/'+hub+'.csv')
df['arrival'] = 1
df = df.groupby(['Date','Hour','Min']).count().reset_index()
df.to_csv(str(granularity)+'min/'+hub+'.csv',index=False)

In [51]:
freq = str(granularity)+'min'
time = pd.date_range('2018-01-01', '2019-01-01',freq=freq)
time = pd.DataFrame(time, columns=['time'])
time = time[:-1]

time['Date'] = time['time'].dt.date.astype('str')
time['Hour'] = time['time'].dt.hour.astype('int')
time['Min'] = time['time'].dt.minute.astype('int')
del time['time']
df = time.merge(df, on=['Date','Hour','Min'], how='left')
df = df.fillna(0)

In [52]:
extDf = pd.read_csv('PENN.csv')
extDf['date'] = pd.to_datetime(extDf['date'], yearfirst=True)
extDf['Hour'] = extDf['date'].dt.hour
extDf['Dow'] = extDf['date'].dt.dayofweek
extDf['Date'] = extDf['date'].dt.date
del extDf['date']
del extDf['arrival']

df.merge(extDf,on=['Hour','Date'],how='left').fillna(0).\
sort_values(by=['Date','Hour','Min']).to_csv(str(granularity)+'min/'+hub+'.csv',index=False)

## Penn

In [16]:
hub = 'PENN'
granularity = 15
df = pd.read_csv('raw2018/'+hub+'.csv',sep='\t',header=None)
df.columns = ['Train#','Origin Date','Sch Ar','Act Ar','Comments','5','6']
del df['5']
del df['6']
df

,Train#,Origin Date,Sch Ar,Act Ar,Comments
0,90,12/31/2018 (Mo),12/31/2018 11:56 PM (Mo),11:53PM,Ar: 3 min early.
1,169,12/31/2018 (Mo),12/31/2018 10:53 PM (Mo),10:47PM,Ar: 6 min early. | Dp: On time.
2,124,12/31/2018 (Mo),12/31/2018 10:49 PM (Mo),10:37PM,Ar: 12 min early.
3,64,12/31/2018 (Mo),12/31/2018 9:55 PM (Mo),9:35PM,Ar: 20 min early.
4,158,12/31/2018 (Mo),12/31/2018 9:45 PM (Mo),9:37PM,Ar: 8 min early.
...,...,...,...,...,...
27289,160,01/01/2018 (Mo),01/01/2018 8:45 AM (Mo),8:51AM,Ar: 6 min late. | Dp: 2 min late.
27290,250,01/01/2018 (Mo),01/01/2018 8:45 AM (Mo),8:48AM,Ar: 3 min late.
27291,150,01/01/2018 (Mo),01/01/2018 6:40 AM (Mo),6:43AM,Ar: 3 min late. | Dp: On time.
27292,65,12/31/2017 (Su),01/01/2018 2:30 AM (Mo),2:40AM,Ar: 10 min late. | Dp: On time.


In [17]:
df = df.dropna()
df['Date'] = df['Sch Ar'].apply(lambda x: x.split(' ')[0])
df['Date'] = pd.to_datetime(df['Date']).astype('str')
# df['Hour'] = df['Act Ar'].apply(lambda x: x.split(' ')[1].split('(')[0])
df['Hour'] = df['Act Ar'].apply(lambda x: 12 + int(x.split(':')[0]) if 'PM' in x else int(x.split(':')[0]))
df['Hour'] = df['Hour'].apply(lambda x: 12 if x==24 else x)
df['Min'] = df['Act Ar'].apply(lambda x: int(x.split(':')[1][:2]))
df['Min'] = (df['Min'] / granularity).astype(int)*granularity
df = df[['Date','Hour','Min']]

In [18]:
freq = str(granularity)+'min'
time = pd.date_range('2018-01-01', '2019-01-01',freq=freq)
time = pd.DataFrame(time, columns=['time'])
time = time[:-1]

time['Date'] = time['time'].dt.date.astype('str')
time['Hour'] = time['time'].dt.hour.astype('int')
time['Min'] = time['time'].dt.minute.astype('int')
del time['time']
df = time.merge(df, on=['Date','Hour','Min'], how='left')
df = df.fillna(0)

In [19]:
df['arrival'] = 1
df = df.groupby(['Date','Hour','Min']).count().reset_index()
extDf = pd.read_csv('PENN.csv')
extDf['date'] = pd.to_datetime(extDf['date'], yearfirst=True)
extDf['Hour'] = extDf['date'].dt.hour.astype('int')
extDf['Dow'] = extDf['date'].dt.dayofweek
extDf['Date'] = extDf['date'].dt.date.astype('str')
del extDf['arrival']
del extDf['date']

df.merge(extDf,on=['Hour','Date'],how='left').fillna(0).\
sort_values(by=['Date','Hour','Min']).to_csv(str(granularity)+'min/'+hub+'.csv',index=False)